# Hello World Example

This is a simple Jupyter Notebook that walks through the 4 steps of compiling and running a PyTorch model on the embedded Neural Processing Unit (NPU) in your AMD Ryzen AI enabled PC. The steps are as follows:

1. Get model
2. Export to ONNX
3. Quantize
4. Run Model on CPU and IPU

In [1]:
# Before starting, be sure you've installed the requirements listed in the requirements.txt file:
!python -m pip install -r requirements.txt

### 1. Get Model
Here, we'll use the PyTorch library to define and instantiate a simple neural network model called `SmallModel`.

In [1]:
import os

print(os.environ['NUM_OF_DPU_RUNNERS'])
os.environ['NUM_OF_DPU_RUNNERS'] = "4"
print(os.environ['NUM_OF_DPU_RUNNERS'])

1
4


In [2]:
import scipy

def generate_win_coeffs(M, P, window_fn="hamming"):
    win_coeffs = scipy.signal.get_window(window_fn, M*P)
    sinc       = scipy.signal.firwin(M * P, cutoff=1.0/P, window="rectangular")
    win_coeffs *= sinc
    return win_coeffs

In [3]:
import torch
import torch.nn as nn
import torch.fft as fft
import numpy as np



class FFTLayer(nn.Module):
    def __init__(self, input_size):
        super(FFTLayer, self).__init__()
        self.input_size = input_size
        self.FFTconv = nn.Conv2d(input_size, (input_size * 2), kernel_size=(1, 1), bias=False)
        #F = torch.zeros((input_size, input_size), dtype=torch.complex128)


        F = torch.from_numpy(np.fft.fft(np.eye(self.input_size)))
        self.FFTconv.weight.data[0:self.input_size ,:,:] = torch.unsqueeze(torch.unsqueeze(F.real.float(), -1), -1)
        self.FFTconv.weight.data[self.input_size:(self.input_size *2),:,:] = torch.unsqueeze(torch.unsqueeze(F.imag.float(), -1), -1)
        #self.FFTconv.weight.requires_grad = False  # Set to `True` if you want to fine-tune the weights




    def forward(self, x):

        output = self.FFTconv(x)



        return output

class PFB_FIR(nn.Module):
    def __init__(self, win_coeffs, M, P, expected_input_size):
        super(PFB_FIR, self).__init__()
        self.win_coeffs = win_coeffs.reshape((M, P)).T
        self.win_coeffs = self.win_coeffs.unsqueeze(0).unsqueeze(1)
        self.win_coeffs = self.win_coeffs.view(P, 1, 1, M)
        self.P = P
        self.M = M
        self.size = expected_input_size
        self.W = int(self.size / self.M / self.P)
        self.Maxsize =  self.M * self.W - self.M
        self.WM = self.M * self.W
        self.FIR = nn.Conv2d(in_channels=self.P, out_channels=self.P, kernel_size=(1, self.M), stride=(1, 1), padding=(0, 0), bias=False, groups=self.P)
        self.FIR.weight = nn.Parameter(self.win_coeffs)
        for param in self.FIR.parameters():
            param.requires_grad = False

    def forward(self, input):
        input = input.view(input.shape[0], self.WM, 1, self.P).permute(0, 3, 2, 1)[:, :, :, 0:self.WM-1]
        out = self.FIR(input)
        return out


class PFB_FIR_FFT(nn.Module):
    def __init__(self, win_coeffs, M, P, expected_input_size):
        super(PFB_FIR_FFT, self).__init__()
        self.win_coeffs = win_coeffs.reshape((M, P)).T
        self.win_coeffs = self.win_coeffs.unsqueeze(0).unsqueeze(1)
        self.win_coeffs = self.win_coeffs.view(P, 1, 1, M)
        self.P = P
        self.M = M
        self.size = expected_input_size
        self.W = int(self.size / self.M / self.P)
        self.Maxsize =  self.M * self.W - self.M
        self.WM = self.M * self.W
        self.FIR = nn.Conv2d(in_channels=self.P, out_channels=self.P, kernel_size=(1, self.M), stride=(1, 1), padding=(0, 0), bias=False, groups=self.P)
        self.FFTlayer = FFTLayer(input_size=self.Maxsize)
        self.FIR.weight = nn.Parameter(self.win_coeffs)
        #for param in self.FIR.parameters():
            #param.requires_grad = False

    def forward(self, input):
        input = input.view(input.shape[0], self.WM, 1, self.P).permute(0, 3, 2, 1)[:, :, :, 0:self.WM-1]
        input = self.FIR(input)
        out = self.FFTlayer(input.view(input.shape[0], self.Maxsize, 1, self.P))
        return out

In [4]:
def createmodel(M, P):
    x = np.sin(np.arange(0, M * P * 10) / np.pi)
    win_coeffs = generate_win_coeffs(M, P, window_fn="hamming")
    
    
    
    
    x = np.random.rand(*x.shape).astype(np.float32)
    
    win_coeffs = torch.from_numpy(win_coeffs)
    
    # Timing pfb_fir_frontend_TINA_FFT
    xinput = np.random.rand(batchsize,*x.shape)
    PFB_layer = PFB_FIR_FFT(win_coeffs = win_coeffs, M = M, P = P, expected_input_size=xinput.shape[1])
    PFB_layer = PFB_layer.float()
    xinput = torch.from_numpy(xinput).float()
    tmp_model_path = "models/pfb.onnx"
    torch.onnx.export(
    PFB_layer,                     # model being run
    xinput,                 # model input (or a tuple for multiple inputs)
    tmp_model_path,                     # where to save the model
    export_params=True,            # store the trained parameter weights inside the model file
    opset_version=13,              # the ONNX version to export the model to
    input_names=['input'],         # the model's input names
    output_names=['output'],       # the model's output names
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}  # variable length axes
    )

    # `input_model_path` is the path to the original, unquantized ONNX model.
    input_model_path = "models/pfb.onnx"
    
    # `output_model_path` is the path where the quantized model will be saved.
    output_model_path = "models/pfb_quantized.onnx"
    
    vai_q_onnx.quantize_static(
    input_model_path,
    output_model_path,
    calibration_data_reader=None,
    quant_format=vai_q_onnx.QuantFormat.QDQ,
    calibrate_method=vai_q_onnx.PowerOfTwoMethod.MinMSE,
    activation_type=vai_q_onnx.QuantType.QUInt8,
    weight_type=vai_q_onnx.QuantType.QInt8,
    enable_ipu_cnn=True,
    extra_options={'ActivationSymmetric': True}
    )
    

In [5]:
def runIPU(input_data):
    # Compile and run

    # Point to the config file path used for the VitisAI Execution Provider
    config_file_path = "vaip_config.json"
    
    #aie_options = onnxruntime.SessionOptions()
    #aie_options.enable_profiling = True
    
    aie_session = onnxruntime.InferenceSession(
        onnx_model_path,
        providers = ['VitisAIExecutionProvider'],
        sess_options=aie_options,
        provider_options=[{'config_file': config_file_path}]
    )
    
    ryzen_outputs = aie_session.run(None, {'input': input_data})
    start = timer()
    ryzen_outputs = aie_session.run(None, {'input': input_data})
    aie_total = timer() - start
    
    #aie_session.end_profiling()
    return aie_total

In [6]:
def pfb_fir_frontend_FFT(x, win_coeffs, M, P):
    #print("it X", x.shape)
    W = int(x.shape[0] / M / P)
    x_p = x.reshape((W*M, P)).T
    h_p = win_coeffs.reshape((M, P)).T
    x_summed = np.zeros((P, M * W - M))
    for t in range(0, M*W-M):
        x_weighted = x_p[:, t:t+M] * h_p
        x_summed[:, t] = x_weighted.sum(axis=1)
    return np.fft.fft(x_summed.T,  axis=1)

In [7]:
import onnxruntime
import numpy as np
from timeit import default_timer as timer

def benchmarkCPU(M,P, win_coeffs):
    #print("M = ", M, " P = ", P)
    # Create some random input data for testing
    input_data_test = np.random.uniform(low=-1, high=1, size=[M * P * 10]).astype(np.float32)
    
    #cpu_options = onnxruntime.SessionOptions()
    #cpu_options.enable_profiling = True
    

    start = timer()
    output = pfb_fir_frontend_FFT(x = input_data_test, win_coeffs = win_coeffs, M = M, P = P)
    cpu_total = timer() - start

    return cpu_total 
    
    #cpu_session.end_profiling()

In [8]:
def comparedspeedup(time1, time2):
  if(time1<= time2):
    return time1
  else:
    return time2

In [9]:
import time


M_increment = 1
P_increment = 16
Min_M = 1
Min_P = 16
Max_M = 30
Max_P = 1024
batchsize = 1
Maxloop = 30

# Placeholder for results
results = []

M = 16
P = 256

x = np.sin(np.arange(0, M * P * 10) / np.pi)
win_coeffs = generate_win_coeffs(M, P, window_fn="hamming")




x = np.random.rand(*x.shape).astype(np.float32)

win_coeffs = torch.from_numpy(win_coeffs)

# Timing pfb_fir_frontend_TINA_FFT
xinput = np.random.rand(batchsize,*x.shape)
PFB_layer = PFB_FIR_FFT(win_coeffs = win_coeffs, M = M, P = P, expected_input_size=xinput.shape[1])
PFB_layer = PFB_layer.float()
xinput = torch.from_numpy(xinput).float()
output = PFB_layer(xinput)

print(output)
print(PFB_layer)

tensor([[[[ 0.2735,  0.3040,  0.2553,  ...,  0.2877,  0.2704,  0.2610]],

         [[-0.0116,  0.0043, -0.0050,  ..., -0.0228,  0.0045,  0.0016]],

         [[ 0.0053,  0.0054, -0.0027,  ...,  0.0122,  0.0165,  0.0135]],

         ...,

         [[ 0.0039, -0.0139,  0.0008,  ...,  0.0071, -0.0109, -0.0027]],

         [[ 0.0005,  0.0097,  0.0006,  ..., -0.0070,  0.0019,  0.0182]],

         [[ 0.0090,  0.0067,  0.0030,  ...,  0.0185, -0.0052,  0.0020]]]],
       grad_fn=<ConvolutionBackward0>)
PFB_FIR_FFT(
  (FIR): Conv2d(256, 256, kernel_size=(1, 16), stride=(1, 1), groups=256, bias=False)
  (FFTlayer): FFTLayer(
    (FFTconv): Conv2d(144, 288, kernel_size=(1, 1), stride=(1, 1), bias=False)
  )
)


In [10]:
# Convert the model to ONNX
tmp_model_path = "models/pfb.onnx"
torch.onnx.export(
    PFB_layer,                     # model being run
    xinput,                 # model input (or a tuple for multiple inputs)
    tmp_model_path,                     # where to save the model
    export_params=True,            # store the trained parameter weights inside the model file
    opset_version=13,              # the ONNX version to export the model to
    input_names=['input'],         # the model's input names
    output_names=['output'],       # the model's output names
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}  # variable length axes
)


In [11]:
import vai_q_onnx

# `input_model_path` is the path to the original, unquantized ONNX model.
input_model_path = "models/pfb.onnx"

# `output_model_path` is the path where the quantized model will be saved.
output_model_path = "models/pfb_quantized.onnx"

vai_q_onnx.quantize_static(
    input_model_path,
    output_model_path,
    calibration_data_reader=None,
    quant_format=vai_q_onnx.QuantFormat.QDQ,
    calibrate_method=vai_q_onnx.PowerOfTwoMethod.MinMSE,
    activation_type=vai_q_onnx.QuantType.QUInt8,
    weight_type=vai_q_onnx.QuantType.QInt8,
    enable_ipu_cnn=True,
    extra_options={'ActivationSymmetric': True}
)

print('Calibrated and quantized model saved at:', output_model_path)

INFO:vai_q_onnx.quantize:calibration_data_reader is None, using random data for calibration
INFO:vai_q_onnx.quant_utils:The input ONNX model models/pfb.onnx can create InferenceSession successfully
INFO:vai_q_onnx.quant_utils:Random input name input shape [1, 40960] type <class 'numpy.float32'> 
INFO:vai_q_onnx.quant_utils:Obtained calibration data with 1 iters
INFO:vai_q_onnx.quant_utils:The input ONNX model models/pfb.onnx can run inference successfully
INFO:vai_q_onnx.quantize:Removed initializers from input
INFO:vai_q_onnx.quantize:Loading model...
INFO:vai_q_onnx.quantize:enable_ipu_cnn is True, optimize the model for better hardware compatibility.
INFO:vai_q_onnx.quantize:Start calibration...
INFO:vai_q_onnx.quantize:Start collecting data, runtime depends on your model size and the number of calibration dataset.
INFO:vai_q_onnx.calibrate:Finding optimal threshold for each tensor using PowerOfTwoMethod.MinMSE algorithm ...
INFO:vai_q_onnx.calibrate:Use all calibration data to calc

[VAI_Q_ONNX_INFO]: Time information:
2024-06-17 15:32:16.677172
[VAI_Q_ONNX_INFO]: OS and CPU information:
                                        system --- Windows
                                          node --- ChrisMiniPC
                                       release --- 10
                                       version --- 10.0.22631
                                       machine --- AMD64
                                     processor --- AMD64 Family 25 Model 116 Stepping 1, AuthenticAMD
[VAI_Q_ONNX_INFO]: Tools version information:
                                        python --- 3.9.19
                                          onnx --- 1.16.1
                                   onnxruntime --- 1.15.1
                                    vai_q_onnx --- 1.16.0+69bc4f2
[VAI_Q_ONNX_INFO]: Quantized Configuration information:
                                   model_input --- models/pfb.onnx
                                  model_output --- models/pfb_quantized.onnx
          

Computing range: 100%|█████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 1381.91tensor/s]
INFO:vai_q_onnx.qdq_quantizer:Remove QuantizeLinear & DequantizeLinear on certain operations(such as conv-relu).


┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Op Type              ┃ Float Model               ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Shape                │ 2                         │
│ Constant             │ 14                        │
│ Gather               │ 2                         │
│ Unsqueeze            │ 2                         │
│ Concat               │ 2                         │
│ Reshape              │ 2                         │
│ Transpose            │ 1                         │
│ Slice                │ 1                         │
│ Conv                 │ 2                         │
├──────────────────────┼───────────────────────────┤
│ Quantized model path │ models/pfb_quantized.onnx │
└──────────────────────┴───────────────────────────┘

Calibrated and quantized model saved at: models/pfb_quantized.onnx


In [12]:
import onnxruntime
import numpy as np
from timeit import default_timer as timer

# Specify the path to the quantized ONNZ Model
onnx_model_path = "models/pfb_quantized.onnx"

# Create some random input data for testing
input_data = np.random.uniform(low=-1, high=1, size=[1,M * P * 10]).astype(np.float32)

cpu_options = onnxruntime.SessionOptions()
cpu_options.enable_profiling = True

# Create Inference Session to run the quantized model on the CPU
cpu_session = onnxruntime.InferenceSession(
    onnx_model_path,
    providers = ['CPUExecutionProvider'],
    sess_options=cpu_options,
)
start = timer()
cpu_results = cpu_session.run(None, {'input': input_data})
cpu_total = timer() - start

cpu_session.end_profiling()


'onnxruntime_profile__2024-06-17_15-32-17.json'

In [13]:
# Compile and run

# Point to the config file path used for the VitisAI Execution Provider
config_file_path = "vaip_config.json"

aie_options = onnxruntime.SessionOptions()
aie_options.enable_profiling = True

aie_session = onnxruntime.InferenceSession(
    onnx_model_path,
    providers = ['VitisAIExecutionProvider'],
    sess_options=aie_options,
    provider_options=[{'config_file': config_file_path}]
)

start = timer()
ryzen_outputs = aie_session.run(None, {'input': input_data})
aie_total = timer() - start

aie_session.end_profiling()

'onnxruntime_profile__2024-06-17_15-32-18.json'

In [14]:
print(f"Ryzen Results: {ryzen_outputs}")
print(f"CPU Results: {cpu_results}")

print(f"CPU Total Time: {cpu_total}")
print(f"IPU Total Time: {aie_total}")

Ryzen Results: [array([[[[ 0.015625  , -0.04296875, -0.046875  , ...,  0.015625  ,
           0.01953125,  0.046875  ]],

        [[ 0.0234375 , -0.01171875,  0.01171875, ...,  0.03515625,
           0.015625  , -0.01953125]],

        [[-0.00390625, -0.03125   ,  0.015625  , ...,  0.02734375,
          -0.0078125 ,  0.00390625]],

        ...,

        [[ 0.01171875,  0.03125   , -0.04296875, ...,  0.0234375 ,
          -0.015625  ,  0.00390625]],

        [[-0.0234375 , -0.015625  ,  0.01171875, ...,  0.015625  ,
           0.0234375 , -0.00390625]],

        [[-0.01171875,  0.0078125 ,  0.        , ...,  0.0078125 ,
           0.0390625 , -0.01171875]]]], dtype=float32)]
CPU Results: [array([[[[ 0.015625  , -0.04296875, -0.046875  , ...,  0.015625  ,
           0.01953125,  0.046875  ]],

        [[ 0.0234375 , -0.01171875,  0.01171875, ...,  0.03515625,
           0.015625  , -0.01953125]],

        [[-0.00390625, -0.03125   ,  0.015625  , ...,  0.02734375,
          -0.0078125 ,  

In [15]:
import time
import numpy as np
import pandas as pd
import torch


M_increment = 3
P_increment = 64
Min_M = 1
Min_P = 64
Max_M = 31
Max_P = 1024
batchsize = 1
Maxloop = 30

# Placeholder for results
results = []

for M in range(Min_M, Max_M + 1, M_increment):
    for P in range(Min_P, Max_P + 1, P_increment):
        createmodel(M, P)
        #print("This is M again: ", M)
        print("This is M: ", M, "This is P: ", P)
        x = np.sin((np.arange(0, M * P * 10) / np.pi),  dtype=np.float32)
        win_coeffs = generate_win_coeffs(M, P, window_fn="hamming")

        # Converting NumPy arrays to CuPy arrays
        
        elapsed_time_FFT = 100
        elapsed_time_TINA_FFT = 100
        bytes_x = x.nbytes
        coef_bytes = win_coeffs.nbytes
        
        for i in range(Maxloop):
            x = np.random.rand(*x.shape).astype(np.float32)

            input_data = np.random.uniform(low=-1, high=1, size=[1,M * P * 10]).astype(np.float32)


          

            # Timing pfb_fir_frontend_FFT
           
            intermediatetime = benchmarkCPU( win_coeffs = win_coeffs, M = M, P = P)
            
            elapsed_time_FFT = comparedspeedup(elapsed_time_FFT, intermediatetime)
            #print("elapsed_time_FFT = " , elapsed_time_FFT)
                        

           
            # Timing pfb_fir_frontend_TINA_FFT
            xinput = np.random.rand(batchsize,*x.shape)
            elapsed_time_TINA_FFT_temp = runIPU(input_data = input_data)
            elapsed_time_TINA_FFT = comparedspeedup(elapsed_time_TINA_FFT, elapsed_time_TINA_FFT_temp)
            #print("elapsed_time_TINA_FFT = " , elapsed_time_TINA_FFT)
            

        # Average the elapsed times
        """
        elapsed_time /= Maxloop
        elapsed_time_FFT /= Maxloop
        elapsed_time_cp /= Maxloop
        elapsed_time_cp_FFT /= Maxloop
        elapsed_time_TINA /= Maxloop
        elapsed_time_TINA_FFT /= Maxloop
        elapsed_time_jax /= Maxloop
        elapsed_time_jax_fft /= Maxloop
        """

         # Calculate speedup for CuPy, Torch, and JAX compared to NumPy
        speedup_TINA_FFT = elapsed_time_FFT / elapsed_time_TINA_FFT
        
        #print("TINA: ", elapsed_time_TINA_FFT)
        #print("cupy: ", elapsed_time_cp_FFT)

        # Append the results to the list
        results.append({
            'Taps': M,
            'Branches': P,
            'bytes_x': bytes_x,
            'coef_bytes': coef_bytes,
            'elapsed_time_FFT_CPU': elapsed_time_FFT,
            'elapsed_time_TINA_FFT': elapsed_time_TINA_FFT,
            'speedup_TINA_FFT':speedup_TINA_FFT,
            

        })

# Create a DataFrame with the results
df = pd.DataFrame(results)

# Divide the values by Maxloop
#df[['elapsed_time', 'elapsed_time_FFT', 'elapsed_time_cp', 'elapsed_time_cp_FFT', 'elapsed_time_TINA', 'elapsed_time_TINA_FFT']] /= Maxloop

# Export the DataFrame to an Excel file
df.to_excel('output PFB.xlsx', index=False)

INFO:vai_q_onnx.quantize:calibration_data_reader is None, using random data for calibration
INFO:vai_q_onnx.quant_utils:The input ONNX model models/pfb.onnx can create InferenceSession successfully
INFO:vai_q_onnx.quant_utils:Random input name input shape [1, 640] type <class 'numpy.float32'> 
INFO:vai_q_onnx.quant_utils:Obtained calibration data with 1 iters
INFO:vai_q_onnx.quant_utils:The input ONNX model models/pfb.onnx can run inference successfully
INFO:vai_q_onnx.quantize:Removed initializers from input
INFO:vai_q_onnx.quantize:Loading model...
INFO:vai_q_onnx.quantize:enable_ipu_cnn is True, optimize the model for better hardware compatibility.
INFO:vai_q_onnx.quantize:Start calibration...
INFO:vai_q_onnx.quantize:Start collecting data, runtime depends on your model size and the number of calibration dataset.
INFO:vai_q_onnx.calibrate:Finding optimal threshold for each tensor using PowerOfTwoMethod.MinMSE algorithm ...
INFO:vai_q_onnx.calibrate:Use all calibration data to calcul

[VAI_Q_ONNX_INFO]: Time information:
2024-06-17 15:32:21.073098
[VAI_Q_ONNX_INFO]: OS and CPU information:
                                        system --- Windows
                                          node --- ChrisMiniPC
                                       release --- 10
                                       version --- 10.0.22631
                                       machine --- AMD64
                                     processor --- AMD64 Family 25 Model 116 Stepping 1, AuthenticAMD
[VAI_Q_ONNX_INFO]: Tools version information:
                                        python --- 3.9.19
                                          onnx --- 1.16.1
                                   onnxruntime --- 1.15.1
                                    vai_q_onnx --- 1.16.0+69bc4f2
[VAI_Q_ONNX_INFO]: Quantized Configuration information:
                                   model_input --- models/pfb.onnx
                                  model_output --- models/pfb_quantized.onnx
          

Computing range: 100%|█████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 3499.00tensor/s]
INFO:vai_q_onnx.qdq_quantizer:Remove QuantizeLinear & DequantizeLinear on certain operations(such as conv-relu).


┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Op Type              ┃ Float Model               ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Shape                │ 2                         │
│ Constant             │ 14                        │
│ Gather               │ 2                         │
│ Unsqueeze            │ 2                         │
│ Concat               │ 2                         │
│ Reshape              │ 2                         │
│ Transpose            │ 1                         │
│ Slice                │ 1                         │
│ Conv                 │ 2                         │
├──────────────────────┼───────────────────────────┤
│ Quantized model path │ models/pfb_quantized.onnx │
└──────────────────────┴───────────────────────────┘

This is M:  1 This is P:  64


...

INFO:vai_q_onnx.quantize:calibration_data_reader is None, using random data for calibration
I0617 17:01:11.610861 16056 quantize.py:164] calibration_data_reader is None, using random data for calibration
INFO:vai_q_onnx.quant_utils:The input ONNX model models/pfb.onnx can create InferenceSession successfully
I0617 17:01:11.615987 16056 quant_utils.py:1232] The input ONNX model models/pfb.onnx can create InferenceSession successfully
INFO:vai_q_onnx.quant_utils:Random input name input shape [1, 317440] type <class 'numpy.float32'> 
I0617 17:01:11.637062 16056 quant_utils.py:507] Random input name input shape [1, 317440] type <class 'numpy.float32'> 
INFO:vai_q_onnx.quant_utils:Obtained calibration data with 1 iters
I0617 17:01:11.642023 16056 quant_utils.py:293] Obtained calibration data with 1 iters
INFO:vai_q_onnx.quant_utils:The input ONNX model models/pfb.onnx can run inference successfully
I0617 17:01:11.651111 16056 quant_utils.py:1211] The input ONNX model models/pfb.onnx can run

[VAI_Q_ONNX_INFO]: Time information:
2024-06-17 17:01:11.596209
[VAI_Q_ONNX_INFO]: OS and CPU information:
                                        system --- Windows
                                          node --- ChrisMiniPC
                                       release --- 10
                                       version --- 10.0.22631
                                       machine --- AMD64
                                     processor --- AMD64 Family 25 Model 116 Stepping 1, AuthenticAMD
[VAI_Q_ONNX_INFO]: Tools version information:
                                        python --- 3.9.19
                                          onnx --- 1.16.1
                                   onnxruntime --- 1.15.1
                                    vai_q_onnx --- 1.16.0+69bc4f2
[VAI_Q_ONNX_INFO]: Quantized Configuration information:
                                   model_input --- models/pfb.onnx
                                  model_output --- models/pfb_quantized.onnx
          

Computing range: 100%|███████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 51.14tensor/s]
INFO:vai_q_onnx.qdq_quantizer:Remove QuantizeLinear & DequantizeLinear on certain operations(such as conv-relu).
I0617 17:01:11.971585 16056 qdq_quantizer.py:886] Remove QuantizeLinear & DequantizeLinear on certain operations(such as conv-relu).


┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Op Type              ┃ Float Model               ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Shape                │ 2                         │
│ Constant             │ 14                        │
│ Gather               │ 2                         │
│ Unsqueeze            │ 2                         │
│ Concat               │ 2                         │
│ Reshape              │ 2                         │
│ Transpose            │ 1                         │
│ Slice                │ 1                         │
│ Conv                 │ 2                         │
├──────────────────────┼───────────────────────────┤
│ Quantized model path │ models/pfb_quantized.onnx │
└──────────────────────┴───────────────────────────┘

This is M:  31 This is P:  1024


In [ ]:
"""
import torch

torch.manual_seed(0)

# Define model class
class SmallModel(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(SmallModel, self).__init__()
        self.fc = torch.nn.Linear(input_size, output_size)

    def forward(self, x):
        output = self.fc(x)
        return output
    

# Instantiate model and generate inputs
input_size = 10
output_size = 5
pytorch_model = SmallModel(input_size, output_size)

print(pytorch_model)
"""

### 2. Export to ONNX

The following code is used for exporting a PyTorch model (pytorch_model) to the ONNX (Open Neural Network Exchange) format. The ONNX file is needed to use the VitisAI Quantizer. 

In [6]:
"""
# Prep for ONNX export
inputs = {"x": torch.rand(input_size, input_size)}
input_names = ['input']
output_names = ['output']
dynamic_axes = {'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
tmp_model_path = "models/pfb.onnx"

# Call export function
torch.onnx.export(
        pytorch_model,
        inputs,
        tmp_model_path,
        export_paradef createmodel(M, P):
    x = np.sin(np.arange(0, M * P * 10) / np.pi)
    win_coeffs = generate_win_coeffs(M, P, window_fn="hamming")
    
    
    
    
    x = np.random.rand(*x.shape).astype(np.float32)
    
    win_coeffs = torch.from_numpy(win_coeffs)
    
    # Timing pfb_fir_frontend_TINA_FFT
    xinput = np.random.rand(batchsize,*x.shape)
    PFB_layer = PFB_FIR_FFT(win_coeffs = win_coeffs, M = M, P = P, expected_input_size=xinput.shape[1])
    PFB_layer = PFB_layer.float()
    xinput = torch.from_numpy(xinput).float()
    tmp_model_path = "models/pfb.onnx"
    torch.onnx.export(
    PFB_layer,                     # model being run
    xinput,                 # model input (or a tuple for multiple inputs)
    tmp_model_path,                     # where to save the model
    export_params=True,            # store the trained parameter weights inside the model file
    opset_version=13,              # the ONNX version to export the model to
    input_names=['input'],         # the model's input names
    output_names=['output'],       # the model's output names
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}  # variable length axes
    )

    # `input_model_path` is the path to the original, unquantized ONNX model.
    input_model_path = "models/pfb.onnx"
    
    # `output_model_path` is the path where the quantized model will be saved.
    output_model_path = "models/pfb_quantized.onnx"
    
    vai_q_onnx.quantize_static(
    input_model_path,
    output_model_path,
    calibration_data_reader=None,
    quant_format=vai_q_onnx.QuantFormat.QDQ,
    calibrate_method=vai_q_onnx.PowerOfTwoMethod.MinMSE,
    activation_type=vai_q_onnx.QuantType.QUInt8,
    weight_type=vai_q_onnx.QuantType.QInt8,
    enable_ipu_cnn=True,
    extra_options={'ActivationSymmetric': True}
    )
    
ms=True,
        opset_version=13,  # Recommended opset
        input_names=input_names,
        output_names=output_names,
        dynamic_axes=dynamic_axes,
    )
"""    

'\n# Prep for ONNX export\ninputs = {"x": torch.rand(input_size, input_size)}\ninput_names = [\'input\']\noutput_names = [\'output\']\ndynamic_axes = {\'input\': {0: \'batch_size\'}, \'output\': {0: \'batch_size\'}}\ntmp_model_path = "models/pfb.onnx"\n\n# Call export function\ntorch.onnx.export(\n        pytorch_model,\n        inputs,\n        tmp_model_path,\n        export_params=True,\n        opset_version=13,  # Recommended opset\n        input_names=input_names,\n        output_names=output_names,\n        dynamic_axes=dynamic_axes,\n    )\n'

### 3. Quantize Model

Using the static quantization method provided by the Vitis AI Quantizer and providing the newly exported ONNX model, we'll quantize the model to INT8. For more information on this quantization method, see [Vitis AI ONNX Quantization](https://ryzenai.docs.amd.com/en/latest/vai_quant/vai_q_onnx.html).

### 4. Run Model

#### CPU Run

Before runnning the model on the IPU, we'll run the model on the CPU and get the execution time for comparison with the IPU. We'll also use the ONNX Runtime Profiling to get some more information about the inference. For more information on this, see [Profiling Tools](https://onnxruntime.ai/docs/performance/tune-performance/profiling-tools.html) from ONNX Runtime. 

#### IPU Run

Now, we'll run it on the IPU and time the execution so that we can compare the results with the CPU.

Let's gather our results and see what we have